# Reading and Cleaning Climate Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [149]:
# cdc wonder Daily Air Temp by County (aggregated by month)
# Jan 1, 1979 - Dec 31, 2011
#temp_df = pd.read_csv('data/climate_data/2011_AirTemp_byState.txt', delimiter="\t", header=0)
#particulateMatter_df = pd.read_csv('data/climate_data/2011_fineParticulateMatter_µgm3.txt', delimiter="\t", header=0)

# Read in Temperature Data

In [2]:
temperature_files = os.listdir('./data/01_climate_data/monthlyAirTemp_byCounty_RAW/')
temperature_files.remove('.ipynb_checkpoints')

In [3]:
temp_1979_df = pd.read_csv('./data/01_climate_data/monthlyAirTemp_byCounty_RAW/dailyAirTemp_byCounty_1979.txt', delimiter='\t', header=0)
temp_1979_df = temp_1979_df[temp_1979_df['County'].isna() == False]
temp_1979_df = temp_1979_df[temp_1979_df['Notes'] != 'Total']

In [4]:
temp_1979_df.columns

Index(['Notes', 'County', 'County Code', 'Month, Year', 'Month, Year Code',
       'Avg Daily Max Air Temperature (F)',
       'Record Count for Daily Max Air Temp (F)',
       'Min Temp for Daily Max Air Temp (F)',
       'Max Temp for Daily Max Air Temp (F)',
       'Avg Daily Min Air Temperature (F)',
       'Record Count for Daily Min Air Temp (F)',
       'Min Temp for Daily Min Air Temp (F)',
       'Max Temp for Daily Min Air Temp (F)', 'Avg Daily Max Heat Index (F)',
       'Record Count for Daily Max Heat Index (F)',
       'Min for Daily Max Heat Index (F)', 'Max for Daily Max Heat Index (F)'],
      dtype='object')

In [153]:
temp_1979_df = temp_1979_df[['County', 'County Code', 'Month, Year', 'Month, Year Code', 'Record Count for Daily Max Air Temp (F)' ,'Avg Daily Max Air Temperature (F)', 'Min Temp for Daily Max Air Temp (F)', 'Max Temp for Daily Max Air Temp (F)']]

In [42]:
# making the above easier with a function to grab data in folder, read it in, remove unwanted stuff and return a clean df
def process_temp_data(filename):
    file_string = './data/01_climate_data/monthlyAirTemp_byCounty_RAW/' + filename
    temp_df = pd.read_csv(file_string, delimiter='\t', header=0, dtype={'County Code': object})
    
    #drop unneccesary rows
    temp_df = temp_df[temp_df['County'].isna() == False]
    temp_df = temp_df[temp_df['Notes'] != 'Total'] #drop the total rows
    
    # create UID
    temp_df['UID'] = temp_df['County'] + " - " + temp_df['Month, Year']
    
    #get only needed columns
    output_df = temp_df[['UID', 'County', 'County Code', 'Month, Year', 'Month, Year Code','Avg Daily Max Air Temperature (F)', 'Min Temp for Daily Max Air Temp (F)', 'Max Temp for Daily Max Air Temp (F)']]
    output_df = output_df.rename(columns={'Avg Daily Max Air Temperature (F)': 'avg_dailyMaxAirTemp_F', 'Min Temp for Daily Max Air Temp (F)': 'min_dailyMaxAirTemp_F', 'Max Temp for Daily Max Air Temp (F)': 'max_dailyMaxAirTemp_F'})
    return output_df

In [43]:
# run process data function on all data in temperature files
air_temperature_df = [process_temp_data(file) for file in temperature_files]
air_temperature_df = pd.concat(air_temperature_df)

# sort the values
air_temperature_df.sort_values(by=['County Code', 'Month, Year Code'], inplace=True)

In [47]:
# turn objects to categories to save memory
air_temperature_df['County'] = air_temperature_df['County'].astype('category')
air_temperature_df['County Code'] = air_temperature_df['County Code'].astype('category')
air_temperature_df['Month, Year'] = air_temperature_df['Month, Year'].astype('category')
air_temperature_df['Month, Year Code'] = air_temperature_df['Month, Year Code'].astype('category')

In [48]:
air_temperature_df.memory_usage(deep=True) / 1_000_000

Index                      9.855648
UID                      107.115228
County                     2.763161
County Code                2.722882
Month, Year                2.506600
Month, Year Code           2.505808
avg_dailyMaxAirTemp_F      9.855648
min_dailyMaxAirTemp_F      9.855648
max_dailyMaxAirTemp_F      9.855648
dtype: float64

In [49]:
air_temperature_df

,UID,County,County Code,"Month, Year","Month, Year Code",avg_dailyMaxAirTemp_F,min_dailyMaxAirTemp_F,max_dailyMaxAirTemp_F
0,"Autauga County, AL - Jan, 1979","Autauga County, AL",01001,"Jan, 1979",1979/01,48.44,29.9,67.1
1,"Autauga County, AL - Feb, 1979","Autauga County, AL",01001,"Feb, 1979",1979/02,54.11,33.5,74.6
2,"Autauga County, AL - Mar, 1979","Autauga County, AL",01001,"Mar, 1979",1979/03,65.65,47.4,78.1
3,"Autauga County, AL - Apr, 1979","Autauga County, AL",01001,"Apr, 1979",1979/04,72.90,62.0,82.7
4,"Autauga County, AL - May, 1979","Autauga County, AL",01001,"May, 1979",1979/05,78.77,66.3,84.5
...,...,...,...,...,...,...,...,...
40437,"Weston County, WY - Aug, 2011","Weston County, WY",56045,"Aug, 2011",2011/08,88.68,69.0,99.4
40438,"Weston County, WY - Sep, 2011","Weston County, WY",56045,"Sep, 2011",2011/09,76.75,45.0,94.0
40439,"Weston County, WY - Oct, 2011","Weston County, WY",56045,"Oct, 2011",2011/10,61.91,33.2,88.7
40440,"Weston County, WY - Nov, 2011","Weston County, WY",56045,"Nov, 2011",2011/11,42.87,17.5,60.0


In [157]:
# #write out clean data to csv
# temp_compression_opts = dict(method='zip',
#                         archive_name='airTempMonthly_1979_2011.csv')


# air_temperature_df.to_csv('./data/01_climate_data/01_climate_data_CLEAN/airTempMonthly_1979_2011.zip', index=False, compression=temp_compression_opts)

In [158]:
# # read in clean csv just to test it works
# data = pd.read_csv('./data/01_climate_data/01_climate_data_CLEAN/airTempMonthly_1979_2011.csv')
# data.info()

# Read in the Particulate Matter Data

In [159]:
# make list of files in particulate matter
particle_files = os.listdir('./data/01_climate_data/monthlyParticulateMatter_RAW/')
#particle_files

In [160]:
# test_pfm_df = pd.read_csv('./data/01_climate_data/monthlyParticulateMatter_RAW/monthlyFPM_byCounty_2009.txt', delimiter="\t", header=0, encoding='')

In [161]:
test_pfm_df.head(2)

,Notes,County,County Code,"Month, Year","Month, Year Code",Avg Fine Particulate Matter (µg/m³),# of Observations for Fine Particulate Matter,Min Fine Particulate Matter,Max Fine Particulate Matter,Avg Fine Particulate Matter Standard Deviation
0,NaN,"Autauga County, AL",1001.0,"Jan, 2009",2009/01,10.61,372.0,3.9,19.2,4.33
1,NaN,"Autauga County, AL",1001.0,"Feb, 2009",2009/02,12.11,336.0,3.7,28.7,5.69


In [162]:
# test_pfm_df = test_pfm_df[test_pfm_df['County'].isna() == False]
# test_pfm_df = test_pfm_df[test_pfm_df['Notes'] != 'Total']

In [163]:
def process_fpm_data(filename):
    file_string = './data/01_climate_data/monthlyParticulateMatter_RAW/' + filename
    fpm_df = pd.read_csv(file_string, delimiter='\t', header=0, encoding='unicode_escape')
    
    #drop unneccesary rows
    fpm_df = fpm_df[fpm_df['County'].isna() == False]
    fpm_df = fpm_df[fpm_df['Notes'] != 'Total'] #drop the total rows
    
    # create UID
    fpm_df['UID'] = fpm_df['County'] + " - " + fpm_df['Month, Year']
    
    #get only needed columns
    output_df = fpm_df[['UID', 'County', 'County Code', 'Month, Year', 'Month, Year Code', 'Avg Fine Particulate Matter (�g/m�)' ,'Min Fine Particulate Matter', 'Max Fine Particulate Matter']]
    final_df = output_df.rename(columns={'Avg Fine Particulate Matter (�g/m�)': 'avg_FPM', 'Min Fine Particulate Matter': 'min_FPM', 'Max Fine Particulate Matter': 'max_FPM'})
    return final_df

In [164]:
#process_fpm_data(particle_files[2]).head(2)

In [165]:
# # run process data function on all data in temperature files
# fine_particulate_matter_df = [process_fpm_data(file) for file in particle_files]
# fine_particulate_matter_df = pd.concat(fine_particulate_matter_df)

# # sort the values
# fine_particulate_matter_df.sort_values(by=['County Code', 'Month, Year Code'], inplace=True)

In [166]:
# # turn objects to categories to save memory
# fine_particulate_matter_df['County'] = fine_particulate_matter_df['County'].astype('category')
# fine_particulate_matter_df['Month, Year'] = fine_particulate_matter_df['Month, Year'].astype('category')
# fine_particulate_matter_df['Month, Year Code'] = fine_particulate_matter_df['Month, Year Code'].astype('category')

In [169]:
#fine_particulate_matter_df

In [170]:
# # creating a Unique Identifier to join later with other climate dfs
# fpm_compression_opts = dict(method='zip',
#                         archive_name='fpm_monthlyByCounty_2003_2011.csv')


# fine_particulate_matter_df.to_csv('./data/01_climate_data/01_climate_data_CLEAN/fpm_monthly_byCounty_2003_2011.zip', index=False, compression=fpm_compression_opts)

# Read in the Heat Wave Days

In [171]:
heat_files = os.listdir('./data/01_climate_data/heatWaveDays_RAW/')
#heat_files

In [172]:
test_df = pd.read_csv('./data/01_climate_data/heatWaveDays_RAW/heatWaveDays_1982_1985.txt', delimiter='\t', header=0)
test_df[test_df['County'].isna() == False].head(1)

,Notes,County,County Code,Year,Year Code,Heat Wave Days Based on Daily Maximum Temperature,Average Heat Wave Days Based on Daily Maximum Temperature,Average Heat Wave Days Based on Daily Maximum Temperature Standard Deviation,Heat Wave Days Based on Daily Maximum Heat Index,Average Heat Wave Days Based on Daily Maximum Heat Index,Average Heat Wave Days Based on Daily Maximum Heat Index Standard Deviation,Heat Wave Days Based on Net Daily Heat Stress,Average Heat Wave Days Based on Net Daily Heat Stress,Average Heat Wave Days Based on Net Daily Heat Stress Standard Deviation,Population,Average Population
0,NaN,"Autauga County, AL",1001.0,1982.0,1982.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,32015,32015.00


In [173]:
test_df.columns

Index(['Notes', 'County', 'County Code', 'Year', 'Year Code',
       'Heat Wave Days Based on Daily Maximum Temperature',
       'Average Heat Wave Days Based on Daily Maximum Temperature',
       'Average Heat Wave Days Based on Daily Maximum Temperature Standard Deviation',
       'Heat Wave Days Based on Daily Maximum Heat Index',
       'Average Heat Wave Days Based on Daily Maximum Heat Index',
       'Average Heat Wave Days Based on Daily Maximum Heat Index Standard Deviation',
       'Heat Wave Days Based on Net Daily Heat Stress',
       'Average Heat Wave Days Based on Net Daily Heat Stress',
       'Average Heat Wave Days Based on Net Daily Heat Stress Standard Deviation',
       'Population', 'Average Population'],
      dtype='object')

In [174]:
def process_heat_wave_data(filename):
    file_string = './data/01_climate_data/heatWaveDays_RAW/' + filename
    hw_df = pd.read_csv(file_string, delimiter='\t', header=0)
    
    #drop unneccesary rows
    hw_df = hw_df[hw_df['County'].isna() == False]
    hw_df = hw_df[hw_df['Notes'] != 'Total'] #drop the total rows
    
    #county code to int
    #hw_df = hw_df.astype({'County Code': 'int64'})
    
    # No UID in this df, since it doesn't have the Month, Year col

    
    #get only needed columns
    output_df = hw_df[['County', 'County Code', 'Year', 'Heat Wave Days Based on Daily Maximum Temperature', 'Heat Wave Days Based on Daily Maximum Heat Index', 'Heat Wave Days Based on Net Daily Heat Stress']]
    final_df = output_df.rename(columns={'Heat Wave Days Based on Daily Maximum Temperature': 'count_hwDays_onDailyMaxTemp', 'Heat Wave Days Based on Daily Maximum Heat Index': 'count_hwDays_onDailyMaxHeatIndex', 'Heat Wave Days Based on Net Daily Heat Stress': 'count_hwDays_onDailyNetHeatStress'})
    return final_df

In [175]:
heat_wave_df = [process_heat_wave_data(file) for file in heat_files]
heat_wave_df = pd.concat(heat_wave_df)
heat_wave_df.sort_values(by=['County Code', 'Year'], inplace=True)

In [176]:
heat_wave_df.to_csv('./data/01_climate_data/01_climate_data_CLEAN/heat_wave_days_1981_2010.csv', index=False)

In [ ]:
# test = pd.read_csv('./data/01_climate_data/01_climate_data_CLEAN/heat_wave_days_1981_2010.csv')
# test.info()

# Read in Precipitation

In [50]:
test_precip = pd.read_csv('./data/01_climate_data/monthlyPrecipitation_RAW/monthlyPrecip_byCounty_1979.txt', delimiter="\t", header=0)
#test_precip

In [51]:
precipitation_files = os.listdir('./data/01_climate_data/monthlyPrecipitation_RAW/')
#precipitation_files

In [52]:
test_precip = test_precip[test_precip['County'].isna() != True]

In [12]:
test_precip = test_precip.astype({'County Code': 'int64'})

In [13]:
test_precip.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40443 entries, 0 to 40442
Data columns (total 9 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Notes                                      3111 non-null   object 
 1   County                                     40443 non-null  object 
 2   County Code                                40443 non-null  int64  
 3   Month, Year                                37332 non-null  object 
 4   Month, Year Code                           37332 non-null  object 
 5   Avg Daily Precipitation (mm)               40443 non-null  float64
 6   # of Observations for Daily Precipitation  40443 non-null  float64
 7   Min Daily Precipitation                    40443 non-null  float64
 8   Max Daily Precipitation                    40443 non-null  float64
dtypes: float64(4), int64(1), object(4)
memory usage: 3.1+ MB


In [53]:
test_precip['UID'] = test_precip['County'] + " - " + test_precip['Month, Year Code']

In [54]:
def process_precip(filename):
    file_string = './data/01_climate_data/monthlyPrecipitation_RAW/' + filename
    precip_df = pd.read_csv(file_string, delimiter='\t', header=0, dtype={'County Code': object})
    
    #drop unneccesary rows
    precip_df = precip_df[precip_df['County'].isna() == False]
    precip_df = precip_df[precip_df['Notes'] != 'Total'] #drop the total rows
    
    #county code to int
    #precip_df = precip_df.astype({'County Code': 'int64'})
    
    # create UID
    precip_df['UID'] = precip_df['County'] + " - " + precip_df['Month, Year']
    
    #get only needed columns
    output_df = precip_df[['UID', 'County', 'County Code', 'Month, Year', 'Month, Year Code', 'Avg Daily Precipitation (mm)' ,'Min Daily Precipitation', 'Max Daily Precipitation']]
    final_df = output_df.rename(columns={'Avg Daily Precipitation (mm)': 'avg_daily_precip_mm', 'Min Daily Precipitation': 'min_daily_precip_mm', 'Max Daily Precipitation': 'max_daily_precip_mm'})
    return final_df

In [55]:
precipitation_df = [process_precip(file) for file in precipitation_files]
precipitation_df = pd.concat(precipitation_df)

In [56]:
precipitation_df.sort_values(by=['County Code', 'Month, Year Code'], inplace=True)

In [57]:
precipitation_df.memory_usage(deep=True) / 1_000_000

Index                    9.855648
UID                    107.115228
County                  92.331756
County Code             76.381272
Month, Year             81.309096
Month, Year Code        78.845184
avg_daily_precip_mm      9.855648
min_daily_precip_mm      9.855648
max_daily_precip_mm      9.855648
dtype: float64

In [41]:
precipitation_df

,UID,County,County Code,"Month, Year","Month, Year Code",avg_daily_precip_mm,min_daily_precip_mm,max_daily_precip_mm
0,"Autauga County, AL - Jan, 1979","Autauga County, AL",01001,"Jan, 1979",1979/01,5.59,0.0,52.6
1,"Autauga County, AL - Feb, 1979","Autauga County, AL",01001,"Feb, 1979",1979/02,5.33,0.0,34.8
2,"Autauga County, AL - Mar, 1979","Autauga County, AL",01001,"Mar, 1979",1979/03,5.72,0.0,124.0
3,"Autauga County, AL - Apr, 1979","Autauga County, AL",01001,"Apr, 1979",1979/04,11.14,0.0,93.7
4,"Autauga County, AL - May, 1979","Autauga County, AL",01001,"May, 1979",1979/05,3.45,0.0,27.2
...,...,...,...,...,...,...,...,...
40437,"Weston County, WY - Aug, 2011","Weston County, WY",56045,"Aug, 2011",2011/08,1.03,0.0,32.9
40438,"Weston County, WY - Sep, 2011","Weston County, WY",56045,"Sep, 2011",2011/09,0.56,0.0,16.3
40439,"Weston County, WY - Oct, 2011","Weston County, WY",56045,"Oct, 2011",2011/10,0.89,0.0,16.3
40440,"Weston County, WY - Nov, 2011","Weston County, WY",56045,"Nov, 2011",2011/11,0.36,0.0,12.1


In [19]:
precipitation_df.columns

Index(['UID', 'County', 'County Code', 'Month, Year', 'Month, Year Code',
       'avg_daily_precip_mm', 'min_daily_precip_mm', 'max_daily_precip_mm'],
      dtype='object')

In [58]:
# turn objects to categories to save memory
precipitation_df['County'] = precipitation_df['County'].astype('category')
precipitation_df['County Code'] = precipitation_df['County Code'].astype('category')
precipitation_df['Month, Year'] = precipitation_df['Month, Year'].astype('category')
precipitation_df['Month, Year Code'] = precipitation_df['Month, Year Code'].astype('category')

In [59]:
precipitation_df.memory_usage(deep=True) / 1_000_000

Index                    9.855648
UID                    107.115228
County                   2.763161
County Code              2.722882
Month, Year              2.506600
Month, Year Code         2.505808
avg_daily_precip_mm      9.855648
min_daily_precip_mm      9.855648
max_daily_precip_mm      9.855648
dtype: float64

In [22]:
precipitation_df

,UID,County,County Code,"Month, Year","Month, Year Code",avg_daily_precip_mm,min_daily_precip_mm,max_daily_precip_mm
29666,"Abbeville County, SC - Jan, 1979","Abbeville County, SC",45001.0,"Jan, 1979",1979/01,5.20,0.0,51.0
29667,"Abbeville County, SC - Feb, 1979","Abbeville County, SC",45001.0,"Feb, 1979",1979/02,6.61,0.0,42.9
29668,"Abbeville County, SC - Mar, 1979","Abbeville County, SC",45001.0,"Mar, 1979",1979/03,2.76,0.0,36.6
29669,"Abbeville County, SC - Apr, 1979","Abbeville County, SC",45001.0,"Apr, 1979",1979/04,6.75,0.0,55.0
29670,"Abbeville County, SC - May, 1979","Abbeville County, SC",45001.0,"May, 1979",1979/05,5.21,0.0,36.9
...,...,...,...,...,...,...,...,...
31116,"Ziebach County, SD - Aug, 2011","Ziebach County, SD",46137.0,"Aug, 2011",2011/08,1.67,0.0,16.9
31117,"Ziebach County, SD - Sep, 2011","Ziebach County, SD",46137.0,"Sep, 2011",2011/09,0.43,0.0,9.9
31118,"Ziebach County, SD - Oct, 2011","Ziebach County, SD",46137.0,"Oct, 2011",2011/10,0.76,0.0,20.8
31119,"Ziebach County, SD - Nov, 2011","Ziebach County, SD",46137.0,"Nov, 2011",2011/11,0.17,0.0,11.9


In [ ]:
# creating a Unique Identifier to join later with other climate dfs
# precip_compression_opts = dict(method='zip',
#                         archive_name='precipitation_monthlyByCounty_1979_2011.csv')

# precipitation_df.to_csv('./data/01_climate_data/01_climate_data_CLEAN/precipitation_monthlyByCounty_1979_2011.zip', compression=precip_compression_opts)

# Combining all Climate Dfs into One

Here we will combine all climate data into one master data frame (sans the heat wave data, since that is not monthly), and write out to a zipped csv. This is to save on file size for github file size limits.

In [60]:
#all dfs to merge: precipitation_df, fine_particulate_matter_df, air_temperature_df
# contextual - heat_wave_df
precip_airTemp = pd.merge(air_temperature_df, precipitation_df, left_on='UID', right_on='UID')
#combined_df = pd.merge(precip_airTemp, fine_particulate_matter_df, left_on='UID', right_on='UID', how='outer')

In [61]:
precip_airTemp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1231956 entries, 0 to 1231955
Data columns (total 15 columns):
 #   Column                 Non-Null Count    Dtype   
---  ------                 --------------    -----   
 0   UID                    1231956 non-null  object  
 1   County_x               1231956 non-null  category
 2   County Code_x          1231956 non-null  category
 3   Month, Year_x          1231956 non-null  category
 4   Month, Year Code_x     1231956 non-null  category
 5   avg_dailyMaxAirTemp_F  1231956 non-null  float64 
 6   min_dailyMaxAirTemp_F  1231956 non-null  float64 
 7   max_dailyMaxAirTemp_F  1231956 non-null  float64 
 8   County_y               1231956 non-null  category
 9   County Code_y          1231956 non-null  category
 10  Month, Year_y          1231956 non-null  category
 11  Month, Year Code_y     1231956 non-null  category
 12  avg_daily_precip_mm    1231956 non-null  float64 
 13  min_daily_precip_mm    1231956 non-null  float64 
 14  ma

In [62]:
precip_airTemp

,UID,County_x,County Code_x,"Month, Year_x","Month, Year Code_x",avg_dailyMaxAirTemp_F,min_dailyMaxAirTemp_F,max_dailyMaxAirTemp_F,County_y,County Code_y,"Month, Year_y","Month, Year Code_y",avg_daily_precip_mm,min_daily_precip_mm,max_daily_precip_mm
0,"Autauga County, AL - Jan, 1979","Autauga County, AL",01001,"Jan, 1979",1979/01,48.44,29.9,67.1,"Autauga County, AL",01001,"Jan, 1979",1979/01,5.59,0.0,52.6
1,"Autauga County, AL - Feb, 1979","Autauga County, AL",01001,"Feb, 1979",1979/02,54.11,33.5,74.6,"Autauga County, AL",01001,"Feb, 1979",1979/02,5.33,0.0,34.8
2,"Autauga County, AL - Mar, 1979","Autauga County, AL",01001,"Mar, 1979",1979/03,65.65,47.4,78.1,"Autauga County, AL",01001,"Mar, 1979",1979/03,5.72,0.0,124.0
3,"Autauga County, AL - Apr, 1979","Autauga County, AL",01001,"Apr, 1979",1979/04,72.90,62.0,82.7,"Autauga County, AL",01001,"Apr, 1979",1979/04,11.14,0.0,93.7
4,"Autauga County, AL - May, 1979","Autauga County, AL",01001,"May, 1979",1979/05,78.77,66.3,84.5,"Autauga County, AL",01001,"May, 1979",1979/05,3.45,0.0,27.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231951,"Weston County, WY - Aug, 2011","Weston County, WY",56045,"Aug, 2011",2011/08,88.68,69.0,99.4,"Weston County, WY",56045,"Aug, 2011",2011/08,1.03,0.0,32.9
1231952,"Weston County, WY - Sep, 2011","Weston County, WY",56045,"Sep, 2011",2011/09,76.75,45.0,94.0,"Weston County, WY",56045,"Sep, 2011",2011/09,0.56,0.0,16.3
1231953,"Weston County, WY - Oct, 2011","Weston County, WY",56045,"Oct, 2011",2011/10,61.91,33.2,88.7,"Weston County, WY",56045,"Oct, 2011",2011/10,0.89,0.0,16.3
1231954,"Weston County, WY - Nov, 2011","Weston County, WY",56045,"Nov, 2011",2011/11,42.87,17.5,60.0,"Weston County, WY",56045,"Nov, 2011",2011/11,0.36,0.0,12.1


In [63]:
precip_airTemp['state_abbrv'] = [i[-2:] for i in precip_airTemp['County_x']]

In [65]:
precip_airTemp.head(2)

,UID,County_x,County Code_x,"Month, Year_x","Month, Year Code_x",avg_dailyMaxAirTemp_F,min_dailyMaxAirTemp_F,max_dailyMaxAirTemp_F,County_y,County Code_y,"Month, Year_y","Month, Year Code_y",avg_daily_precip_mm,min_daily_precip_mm,max_daily_precip_mm,state_abbrv
0,"Autauga County, AL - Jan, 1979","Autauga County, AL",01001,"Jan, 1979",1979/01,48.44,29.9,67.1,"Autauga County, AL",01001,"Jan, 1979",1979/01,5.59,0.0,52.6,AL
1,"Autauga County, AL - Feb, 1979","Autauga County, AL",01001,"Feb, 1979",1979/02,54.11,33.5,74.6,"Autauga County, AL",01001,"Feb, 1979",1979/02,5.33,0.0,34.8,AL


In [66]:
precip_airTemp.drop(columns=['UID','County_y', 'County Code_y', 'Month, Year_y', 'Month, Year Code_y'], inplace=True)

In [67]:
precip_airTemp.rename(columns={'County_x': 'county_name', 'County Code_x': 'FIPS', 'Month, Year_x': 'month_year_long', 'Month, Year Code_x': 'month_year_short'}, inplace=True)

In [68]:
precip_airTemp['state_abbrv'] = precip_airTemp['state_abbrv'].astype('category')

In [69]:
# creating a Unique Identifier to join later with other climate dfs
final_compression_opts = dict(method='zip',
                        archive_name='cprecip_airTemp_1979_2011.csv')

precip_airTemp.to_csv('./data/01_climate_data/01_climate_data_CLEAN/precip_airTemp_wFIPS_1979_2011.zip', compression=final_compression_opts)

In [70]:
precip_airTemp.memory_usage(deep=True).sum() / 1_000_000

80.723906

In [73]:
len(precip_airTemp[precip_airTemp['state_abbrv'] == 'TX']['FIPS'].unique())

254

In [ ]:
combined_df

In [ ]:
combined_df.memory_usage(deep=True) / 1_000_000

# Reading in Marcus' Health Data

tricky, but I think I can do it

In [47]:
cvd_files = os.listdir('./data/03_health_data/IHME_USA_COUNTY_CVD_MORTALITY_RATES_1980_2014/')
inf_files = os.listdir('./data/03_health_data/IHME_USA_COUNTY_INFECT_DIS_MORT_1980_2014/')
resp_files = os.listdir('./data/03_health_data/IHME_USA_COUNTY_RESP_DISEASE_MORTALITY_1980_2014/')
cancer_files = os.listdir('./data/03_health_data/HME_USA_COUNTY_CANCER_MORTALITY_RATES_1980_2014/')
substance_injury_files = os.listdir('./data/03_health_data/IHME_USA_COUNTY_USE_INJ_MORTALITY_1980_2014/')

#obesity_files = os.listdir('./data/IHME_USA_OBESITY_PHYSICAL_ACTIVITY_2001_2011.csv')
cvd_files.remove('.ipynb_checkpoints')
inf_files.remove('.ipynb_checkpoints')
#resp_files.remove('.ipynb_checkpoints')
#cancer_files.remove('.ipynb_checkpoints')
#substance_injury_files.remove('.ipynb_checkpoints')

In [ ]:
substance_injury_files

In [ ]:
# states = []
# for file in cvd_files:
#     state = file.split('_')[-2]
#     states.append(state.title())

In [ ]:
# # test a workflow for cleaning on state's file
# filename = 'IHME_USA_COUNTY_CVD_MORTALITY_RATES_1980_2014_' + states[10].upper() + '_Y2017M05D16.CSV'
# filepath = './data/IHME_USA_COUNTY_CVD_MORTALITY_RATES_1980_2014/' + filename
# #print(filepath)
# test = pd.read_csv(filepath)
# test = test[test['location_name'] != states[10]]

# test[['FIPS', 'cause_id', 'sex_id', 'year_id']] = test[['FIPS', 'cause_id', 'sex_id', 'year_id']].astype('string')
# test['UID'] = test['FIPS'] + '-' + test['cause_id'] + '-' + test['sex_id'] + '-' + test['year_id']

In [48]:
# universal function to process data in all folders
def process_health_data(file):
    state = file.split('_')[-2].title()
    stat_type = file.split('_')[3]
    
    #set filepath based on stat_type
    if stat_type == 'CVD':
        folder = './data/03_health_data/IHME_USA_COUNTY_CVD_MORTALITY_RATES_1980_2014/'
    elif stat_type == 'INFECT':
        folder = './data/03_health_data/IHME_USA_COUNTY_INFECT_DIS_MORT_1980_2014/'
    elif stat_type == 'RESP':
        folder = './data/03_health_data/IHME_USA_COUNTY_RESP_DISEASE_MORTALITY_1980_2014/'
    elif stat_type == 'CANCER':
        folder = './data/03_health_data/HME_USA_COUNTY_CANCER_MORTALITY_RATES_1980_2014/'
    elif stat_type == 'USE':
        folder = './data/03_health_data/IHME_USA_COUNTY_USE_INJ_MORTALITY_1980_2014/'
    full_path = folder + file
    #print(full_path)
    
    #read in csv
    df = pd.read_csv(full_path)
    # drop rows that are sums of whole state
    df = df[df['location_name'] != state]
    
    #recase cols to save memory and so we can create a UID later
    # UID is formatted: 'FIPS-cause_id-sex_id-year_id'
    df[['FIPS', 'cause_id', 'sex_id', 'year_id']] = df[['FIPS', 'cause_id', 'sex_id', 'year_id']].astype('string')
    df['UID'] = df['FIPS'] + '-' + df['cause_id'] + '-' + df['sex_id'] + '-' + df['year_id']
    
    #recast variables as category for file size handling
    #df[['location_name', 'FIPS', 'cause_id', 'cause_name', 'sex_id', 'sex', 'year_id', 'UID']] = df[['location_name', 'FIPS', 'cause_id', 'cause_name', 'sex_id', 'sex', 'year_id', 'UID']].astype('category')
    
    return df

In [91]:
process_health_data(cancer_files[3]).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278390 entries, 105 to 281784
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   location_id    278390 non-null  int64   
 1   location_name  278390 non-null  category
 2   FIPS           278390 non-null  category
 3   cause_id       278390 non-null  category
 4   cause_name     278390 non-null  category
 5   sex_id         278390 non-null  category
 6   sex            278390 non-null  category
 7   year_id        278390 non-null  category
 8   mx             278390 non-null  float64 
 9   upper          278390 non-null  float64 
 10  lower          278390 non-null  float64 
 11  UID            278390 non-null  category
dtypes: category(8), float64(3), int64(1)
memory usage: 23.7 MB


In [139]:
# process cardiovascular mortality data
full_cvd_df = [process_health_data(file) for file in cvd_files]
full_cvd_df = pd.concat(full_cvd_df)

#recast variables as category for file size handling
full_cvd_df[['location_name', 'FIPS', 'cause_id', 'cause_name', 'sex_id', 'sex', 'year_id', 'UID']] = full_cvd_df[['location_name', 'FIPS', 'cause_id', 'cause_name', 'sex_id', 'sex', 'year_id', 'UID']].astype('category')

# full_cvd_df_1980 = full_cvd_df[full_cvd_df['year_id'] == '1980']
# full_cvd_df_1980[:50]

In [140]:
full_cvd_df.head(3)

,measure_id,measure_name,location_id,location_name,FIPS,cause_id,cause_name,sex_id,sex,age_id,age_name,year_id,metric,mx,upper,lower,UID
105,1,Deaths,2222,Beaverhead County,30001,491,Cardiovascular diseases,1,Male,27,Age-standardized,1980,Rate,629.352118,704.298390,562.459255,30001-491-1-1980
106,1,Deaths,2222,Beaverhead County,30001,491,Cardiovascular diseases,1,Male,27,Age-standardized,1981,Rate,622.091399,692.320932,562.103676,30001-491-1-1981
107,1,Deaths,2222,Beaverhead County,30001,491,Cardiovascular diseases,1,Male,27,Age-standardized,1982,Rate,590.455653,653.424627,533.480891,30001-491-1-1982


In [141]:
# get cvd cause ids
cvd_cause_ids = list(full_cvd_df['cause_id'].unique())
cvd_cause_names = list(full_cvd_df['cause_name'].unique())
cvd_causes= {cvd_cause_ids[i]: cvd_cause_names[i] for i in range(len(cvd_cause_ids))}
cvd_causes

{'491': 'Cardiovascular diseases',
 '492': 'Rheumatic heart disease',
 '493': 'Ischemic heart disease',
 '494': 'Cerebrovascular disease',
 '495': 'Ischemic stroke',
 '496': 'Hemorrhagic stroke',
 '498': 'Hypertensive heart disease',
 '499': 'Cardiomyopathy and myocarditis',
 '500': 'Atrial fibrillation and flutter',
 '501': 'Aortic aneurysm',
 '502': 'Peripheral vascular disease',
 '503': 'Endocarditis',
 '507': 'Other cardiovascular and circulatory diseases'}

In [130]:
#full_cvd_df = full_cvd_df[full_cvd_df['sex'] == 'Both']

In [142]:
full_cvd_df.drop(columns=['measure_id', 'location_id', 'measure_name', 'age_name', 'metric', 'UID', 'age_id', 'upper', 'lower', 'sex_id'], inplace=True)

In [143]:
full_cvd_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4290195 entries, 105 to 121484
Data columns (total 7 columns):
 #   Column         Dtype   
---  ------         -----   
 0   location_name  category
 1   FIPS           category
 2   cause_id       category
 3   cause_name     category
 4   sex            category
 5   year_id        category
 6   mx             float64 
dtypes: category(6), float64(1)
memory usage: 98.4 MB


In [261]:
full_cvd_df['FIPS'].unique()

['30001', '30003', '30005', '30007', '30009', ..., '39167', '39169', '39171', '39173', '39175']
Length: 3143
Categories (3143, object): ['10001', '10003', '10005', '1001', ..., '9009', '9011', '9013', '9015']

In [263]:
full_cvd_df.memory_usage(deep=True) / 1_000_000 #get mb

Index            34.321560
location_name     8.780426
FIPS              8.841025
cause_id          4.291531
cause_name        4.291802
sex               4.290488
year_id           4.293402
mx               34.321560
dtype: float64

In [146]:
# creating a Unique Identifier to join later with other climate dfs
cvd_df_compression_opts = dict(method='zip',
                        archive_name='full_cvd_df.csv')

full_cvd_df.to_csv('./data/03_health_data/01_health_data_CLEAN/cvd_mortality.zip', compression=cvd_df_compression_opts)

In [72]:
# process infectious disease mortality data
full_inf_df = [process_health_data(file) for file in inf_files]
full_inf_df = pd.concat(full_inf_df)

In [76]:
inf_cause_ids = list(full_inf_df['cause_id'].unique())
inf_cause_names = list(full_inf_df['cause_name'].unique())
inf_causes = {inf_cause_ids[i]: inf_cause_names[i] for i in range(len(inf_cause_ids))}
inf_causes

{'297': 'Tuberculosis',
 '298': 'HIV/AIDS',
 '302': 'Diarrheal diseases',
 '322': 'Lower respiratory infections',
 '332': 'Meningitis',
 '400': 'Hepatitis'}

In [78]:
full_inf_df.memory_usage(deep=True) / 1_000_000 #get mb

Index             15.896160
measure_id        15.896160
measure_name     125.182260
location_id       15.896160
location_name    141.150240
FIPS             122.956470
cause_id           1.987552
cause_name         1.987619
sex_id             1.987302
sex                1.987313
age_id            15.896160
age_name         145.052460
year_id            1.990227
metric           121.208220
mx                15.896160
lower             15.896160
upper             15.896160
UID              144.806760
dtype: float64

In [77]:
# process respiratory disease mortality data
full_resp_df = [process_health_data(file) for file in resp_files]
full_resp_df = pd.concat(full_resp_df)

In [90]:
full_resp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3311700 entries, 105 to 108149
Data columns (total 17 columns):
 #   Column         Dtype   
---  ------         -----   
 0   measure_id     int64   
 1   measure_name   object  
 2   location_id    int64   
 3   location_name  object  
 4   FIPS           object  
 5   cause_id       category
 6   cause_name     category
 7   sex_id         category
 8   sex            category
 9   age_id         int64   
 10  age_name       object  
 11  year_id        category
 12  metric         object  
 13  mx             float64 
 14  lower          float64 
 15  upper          float64 
 16  UID            object  
dtypes: category(5), float64(3), int64(3), object(6)
memory usage: 344.3+ MB


In [82]:
resp_cause_ids = list(full_resp_df['cause_id'].unique())
resp_cause_names = list(full_resp_df['cause_name'].unique())
resp_causes = {resp_cause_ids[i]: resp_cause_names[i] for i in range(len(resp_cause_ids))}
resp_causes

{'508': 'Chronic respiratory diseases',
 '509': 'Chronic obstructive pulmonary disease',
 '510': 'Pneumoconiosis',
 '511': 'Silicosis',
 '512': 'Asbestosis',
 '513': 'Coal workers pneumoconiosis',
 '514': 'Other pneumoconiosis',
 '515': 'Asthma',
 '516': 'Interstitial lung disease and pulmonary sarcoidosis',
 '520': 'Other chronic respiratory diseases'}

In [83]:
full_resp_df.memory_usage(deep=True) / 1_000_000 #get mb

Index             26.493600
measure_id        26.493600
measure_name     208.637100
location_id       26.493600
location_name    235.250400
FIPS             204.927450
cause_id           3.312600
cause_name         3.312806
sex_id             3.311982
sex                3.311993
age_id            26.493600
age_name         241.754100
year_id            3.314907
metric           202.013700
mx                26.493600
lower             26.493600
upper             26.493600
UID              241.344600
dtype: float64

In [84]:
# process cancer mortality data
full_cancer_df = [process_health_data(file) for file in cancer_files]
full_cancer_df = pd.concat(full_cancer_df)

In [86]:
cancer_cause_ids = list(full_cancer_df['cause_id'].unique())
cancer_cause_names = list(full_cancer_df['cause_name'].unique())
cancer_causes = {cancer_cause_ids[i]: cancer_cause_names[i] for i in range(len(cancer_cause_ids))}
cancer_causes

{'410': 'Neoplasms',
 '444': 'Lip and oral cavity cancer',
 '447': 'Nasopharynx cancer',
 '450': 'Other pharynx cancer',
 '411': 'Esophageal cancer',
 '414': 'Stomach cancer',
 '441': 'Colon and rectum cancer',
 '417': 'Liver cancer',
 '453': 'Gallbladder and biliary tract cancer',
 '456': 'Pancreatic cancer',
 '423': 'Larynx cancer',
 '426': 'Tracheal, bronchus, and lung cancer',
 '459': 'Malignant skin melanoma',
 '462': 'Non-melanoma skin cancer',
 '429': 'Breast cancer',
 '432': 'Cervical cancer',
 '435': 'Uterine cancer',
 '465': 'Ovarian cancer',
 '438': 'Prostate cancer',
 '468': 'Testicular cancer',
 '471': 'Kidney cancer',
 '474': 'Bladder cancer',
 '477': 'Brain and nervous system cancer',
 '480': 'Thyroid cancer',
 '483': 'Mesothelioma',
 '484': 'Hodgkin lymphoma',
 '485': 'Non-Hodgkin lymphoma',
 '486': 'Multiple myeloma',
 '487': 'Leukemia',
 '845': 'Acute lymphoid leukemia',
 '846': 'Chronic lymphoid leukemia',
 '847': 'Acute myeloid leukemia',
 '848': 'Chronic myeloid le

In [89]:
full_cancer_df[['location_name']]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10670485 entries, 105 to 410794
Data columns (total 12 columns):
 #   Column         Dtype  
---  ------         -----  
 0   location_id    int64  
 1   location_name  object 
 2   FIPS           object 
 3   cause_id       object 
 4   cause_name     object 
 5   sex_id         object 
 6   sex            object 
 7   year_id        object 
 8   mx             float64
 9   upper          float64
 10  lower          float64
 11  UID            object 
dtypes: float64(3), int64(1), object(8)
memory usage: 1.0+ GB


In [87]:
full_cancer_df.memory_usage(deep=True) / 1_000_000 #get mb

Index             85.363880
location_id       85.363880
location_name    758.069550
FIPS             660.487065
cause_id         640.229100
cause_name       807.216690
sex_id           618.888130
sex              657.939905
year_id          650.899585
mx                85.363880
upper             85.363880
lower             85.363880
UID              777.862400
dtype: float64

In [88]:
# process substance use / self injury mortality data
full_substances_inj_df = [process_health_data(file) for file in substance_injury_files]
full_substances_inj_df = pd.concat(full_substances_inj_df)

In [ ]:
sub_inj_ids = list(full)

In [662]:
full_substances_inj_df.memory_usage(deep=True) / 1_000_000 #get mb

Index            10.597440
measure_id       10.597440
measure_name     83.454840
location_id      10.597440
location_name    94.100160
FIPS             81.970980
cause_id          1.325092
cause_name        1.325150
sex_id            1.324962
sex               1.324973
age_id           10.597440
age_name         96.701640
year_id           1.327887
metric           80.805480
mx               10.597440
lower            10.597440
upper            10.597440
UID              96.537840
measure_ID       10.597440
dtype: float64